## Dog Breed Classification

In this project we will use traditional CNN, CNN with data augmentation and finally transfer Learning by VGG16 model with weights pre-trained on Imagenet to solve the dog breed classification problem

### Load Dataset Files

In [0]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
import pandas as pd
import numpy as np

Using TensorFlow backend.


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Now, upload the given dataset file shared with you in your google drive and give its path for the below given `project_path` variable. For example, a path is given below according to the file path in our google drive. You need to change this to match the path of yours.

In [0]:
project_path = "/content/drive/My Drive/dog/"

In [0]:
import os
os.chdir('/content/drive/My Drive/dog/')

Run the below code to extract all the images in the train.zip files given in the dataset. We are going to use these images as train and validation sets and their labels in further steps.


In [0]:
# from zipfile import ZipFile
# with ZipFile(project_path+'train.zip', 'r') as z:
#   z.extractall()

Repeat the same step for test.zip

In [0]:
# from zipfile import ZipFile
# with ZipFile(project_path+'test.zip', 'r') as z:
#   z.extractall()

Repeat the same step for sample_submission.csv.zip

In [0]:
# from zipfile import ZipFile
# with ZipFile(project_path+'sample_submission.csv.zip', 'r') as z:
#   z.extractall()

Repeat the same step for labels.csv.zip

In [0]:
# from zipfile import ZipFile
# with ZipFile(project_path+'labels.csv.zip', 'r') as z:
#   z.extractall()

After this process, we will have 4 files - Train folder, test folder and labels.csv and sample_submission.csv as part of your google drive

### Read labels.csv file using pandas

In [0]:
labels = pd.read_csv(project_path+"labels.csv")
labels.head()

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


In [0]:
labels.breed.nunique()

120

### Print the count of each category of Dogs given in the dataset



In [0]:
print("Number of dog breeds are:",labels.breed.nunique())

Number of dog breeds are: 120


In [0]:
print(labels.breed.value_counts().head())

scottish_deerhound      126
maltese_dog             117
afghan_hound            116
entlebucher             115
bernese_mountain_dog    114
Name: breed, dtype: int64


In [0]:
print(labels.breed.value_counts().tail())

brabancon_griffon    67
komondor             67
golden_retriever     67
eskimo_dog           66
briard               66
Name: breed, dtype: int64


So there 120 diffrent breads of dogs,So now our task is to to classify a given image into one of these 120 images.This is a multi class classification problem.

### Get one-hot encodings of labels

In [0]:
breed_Ohe=np.array(pd.get_dummies(labels.breed))

## Preparing training dataset
1. Write a code which reads each and every id from labels.csv file and loads the corresponding image (in RGB - 128, 128, 3) from the train folder. <br>
2. Create 2 variables <br> 
     a.  x_train - Should have all the images of the dogs from train folder <br>
     b.  y_train - Corresponding label of the dog <br>
<u>Note:</u> The id of the dog images and its corresponding labels are available in labels.csv file   
<u>Hint:</u> Watch the video shared on "Preparing the training dataset" if you face issue on creating the training dataset

In [1]:
from os import listdir, makedirs
from os.path import join, exists, expanduser
print(len(listdir(join(project_path, 'train'))))
print(len(labels))
print(len(listdir(join(project_path, 'test'))))

5000
10222
10357


In [0]:
train_img=labels.id.values

In [0]:
import tqdm
import cv2
x_train_feature = []
i = 0 # initialisation
for f in train_img: # f for format ,jpg
    img = cv2.imread('./train/{}.jpg'.format(f), 0)
    img_resize = cv2.resize(img, (200, 200))
    # normalized images 
    img_resize=(img_resize - img_resize.mean()) / img_resize.std()
    x_train_feature.append(img_resize)


In [0]:
x_train_feature=np.array(x_train_feature)
x_train = x_train_feature.reshape(x_train_feature.shape[0], 200,200, 1)

Normalize the training data and convert into 4 dimensions so that it can be used as an input to conv layers in the model

In [0]:
#we normalized images in the avbove step itself

### Split the training and validation data from `x_train_data` and `y_train_data` obtained from above step

In [0]:
np.random.seed(seed=123)
rnd = np.random.random(len(labels))
train_idx = rnd < 0.8
valid_idx = rnd >= 0.8
x_tr=np.array(x_train)[rnd < 0.8]
x_cv=np.array(x_train)[valid_idx]
y_tr=breed_Ohe[train_idx]
y_cv=breed_Ohe[valid_idx]
print(x_tr.shape)
print(x_cv.shape)
print(y_tr.shape)
print(y_cv.shape)

(4006, 200, 200, 3)
(994, 200, 200, 3)
(4006, 120)
(994, 120)


### Loading the test data
Read the id column from the samples_submission.csv and store it in test_img

In [0]:
sample_submission = pd.read_csv(project_path+"sample_submission.csv")[0:5000]

In [0]:
test_img=sample_submission.id.values  

In [0]:
sample_submission.id.nunique()

Run the below code to load the test image files in x_test_feature

In [0]:
x_test_feature = []
i = 0 # initialisation
for f in test_img: # f for format ,jpg
    img = cv2.imread('./test/{}.jpg'.format(f), 0)
    img_resize = cv2.resize(img, (224, 224)) 
    # normalized images
    img_resize=(img_resize - img_resize.mean()) / img_resize.std()
    x_test_feature.append(img_resize)

Normalize the test data and convert it into 4 dimensions

In [0]:
#we normalized images in the avbove step itself

### Build a basic conv neural network with 2 conv layers (kernel sizes - 5 and 3) add layers as mentioned below for classification.

1. Add a Dense layer with 256 neurons with `relu` activation

2. Add a Dense layer with 120 neurons as final layer (as there are 120 classes in the given dataset) with `softmax` activation for classifiaction. 

In [0]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(5, 5),activation='relu',input_shape=(200,200,1)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(120, activation='relu'))
model.add(Dense(120, activation='softmax'))
model.summary()





Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 196, 196, 32)      832       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 194, 194, 32)      9248      
_________________________________________________________________
flatten_1 (Flatten)          (None, 1204352)           0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               308314368 
_________________________________________________________________
dense_2 (Dense)              (None, 120)               30840     
_________________________________________________________________
dense_3 (Dense)              (None, 120)               14520     
Total params: 308,369,808
Trainable params: 308,369,808
Non-trainable params: 0
_____________________________________

In [0]:
model.compile(loss=keras.losses.categorical_crossentropy,
optimizer=keras.optimizers.Adadelta(),
metrics=['accuracy'])

### Use batch_size = 128 and epochs = 10 and execute the model

In [0]:
history = model.fit(x_tr, y_tr,
batch_size=128,
epochs=10,
verbose=1,
validation_data=(x_cv, y_cv))
score = model.evaluate(x_cv, y_cv, verbose=0)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 4006 samples, validate on 994 samples
Epoch 1/10





4006/4006 [==============================] - 22s 5ms/step - loss: 5.6484 - acc: 0.0147 - val_loss: 4.8192 - val_acc: 0.0111
Epoch 2/10
4006/4006 [==============================] - 10s 2ms/step - loss: 4.6466 - acc: 0.0334 - val_loss: 4.7433 - val_acc: 0.0191
Epoch 3/10
4006/4006 [==============================] - 10s 2ms/step - loss: 4.2767 - acc: 0.1008 - val_loss: 5.0782 - val_acc: 0.0151
Epoch 4/10
4006/4006 [==============================] - 10s 2ms/step - loss: 3.4899 - acc: 0.2746 - val_loss: 6.4615 - val_acc: 0.0201
Epoch 5/10
4006/4006 [==============================] - 10s 2ms/step - loss: 1.4319 - acc: 0.7000 - val_loss: 7.4659 - val_acc: 0.0101
Epoch 6/10
4006/4006 [==============================] - 10s 2ms/step - loss: 0.0963 - acc: 0.9908 - val_loss: 8.1429 - val_acc: 0.0161
Epoch 7/10
4006/4006 [===================

#The model accuracy is very poor !!!!

### Use Data Augmentation in the above model to see if the accuracy improves


In [0]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

### Using the above objects, create the image generators with variable names `train_generator` and `val_generator`

You need to use train_datagen.flow() and val_datagen.flow()

In [0]:

train_generator = train_datagen.flow(x_tr, y_tr, batch_size=128)

validation_generator = test_datagen.flow(x_cv, y_cv, batch_size=128)

In [0]:
model.compile(loss=keras.losses.categorical_crossentropy,
optimizer=keras.optimizers.Adadelta(),
metrics=['accuracy'])

### Fit the model using fit_generator() using `train_generator` and `val_generator` from the above step with 10 epochs

In [0]:
model.fit_generator(
        train_generator,
        epochs=10,
        validation_data=validation_generator)

Epoch 1/10
32/32 [==============================] - 12s 360ms/step - loss: 4.7747 - acc: 0.0139 - val_loss: 4.7873 - val_acc: 0.0131
Epoch 2/10
32/32 [==============================] - 11s 348ms/step - loss: 4.7745 - acc: 0.0122 - val_loss: 4.7881 - val_acc: 0.0131
Epoch 3/10
32/32 [==============================] - 11s 340ms/step - loss: 4.7740 - acc: 0.0122 - val_loss: 4.7889 - val_acc: 0.0131
Epoch 4/10
32/32 [==============================] - 11s 330ms/step - loss: 4.7736 - acc: 0.0122 - val_loss: 4.7896 - val_acc: 0.0131
Epoch 5/10
32/32 [==============================] - 11s 335ms/step - loss: 4.7728 - acc: 0.0122 - val_loss: 4.7902 - val_acc: 0.0131
Epoch 6/10
32/32 [==============================] - 11s 335ms/step - loss: 4.7719 - acc: 0.0128 - val_loss: 4.7906 - val_acc: 0.0131
Epoch 7/10
32/32 [==============================] - 11s 334ms/step - loss: 4.7720 - acc: 0.0115 - val_loss: 4.7912 - val_acc: 0.0131
Epoch 8/10
32/32 [==============================] - 11s 336ms/step - 

# Model accuracy is still poor!!!

### Lets use Transfer Learning

Download the vgg wieght file from here : https://github.com/MinerKasch/applied_deep_learning/blob/master/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5

Use the below code to load VGG16 weights trained on ImageNet

In [0]:
from keras.applications.vgg16 import VGG16, preprocess_input
# Instantiate the model with the pre-trained weights (no top)
base_model= VGG16(weights=(project_path+'vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'),
                 include_top=False, pooling='avg')

Print the summary of the base_model

In [0]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0     

In [0]:
base_model.trainable = False

### Add the following classification layers to the imported VGG Model <br>
1. Flatten Layer
2. Dense layer with 1024 neurons with activation as Relu
3. Dense layer with 256 neurons with activation as Relu
4. Dense layer with 120 neurons with activation as Softmax

In [0]:
#here the images have three channels, while running this model i read the data once again with three chanells,i just executed the data preparation code once again twith the second parameter in imread() set to 1 

from keras.models import Model
base_model.trainable = False
x = (base_model.output)
x = Dense(1024, activation='relu')(x)
x = Dense(256, activation='relu')(x)
predictions = Dense(120, activation = 'softmax')(x)
head_model = Model(input = base_model.input, output = predictions)
head_model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

head_model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0   

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  import sys


### Make all the layers in the base_model (VGG16) to be non-trainable

In [0]:
# Freeze the layers except the last 4 layers
for layer in head_model.layers[:-4]:
    layer.trainable = False
 
# Check the trainable status of the individual layers
for layer in head_model.layers:
    print(layer, layer.trainable)

<keras.engine.input_layer.InputLayer object at 0x7f6e5b24afd0> False
<keras.layers.convolutional.Conv2D object at 0x7f6e5b1a7be0> False
<keras.layers.convolutional.Conv2D object at 0x7f6e5b214f98> False
<keras.layers.pooling.MaxPooling2D object at 0x7f6ed7296240> False
<keras.layers.convolutional.Conv2D object at 0x7f6e5b1be5c0> False
<keras.layers.convolutional.Conv2D object at 0x7f6e5b1c48d0> False
<keras.layers.pooling.MaxPooling2D object at 0x7f6e5b1c7438> False
<keras.layers.convolutional.Conv2D object at 0x7f6e5b1cfd68> False
<keras.layers.convolutional.Conv2D object at 0x7f6e5b1d5e48> False
<keras.layers.convolutional.Conv2D object at 0x7f6e5b1dbda0> False
<keras.layers.pooling.MaxPooling2D object at 0x7f6e5b16b358> False
<keras.layers.convolutional.Conv2D object at 0x7f6e5b170c88> False
<keras.layers.convolutional.Conv2D object at 0x7f6e5b177d68> False
<keras.layers.convolutional.Conv2D object at 0x7f6e5b17dcc0> False
<keras.layers.pooling.MaxPooling2D object at 0x7f6e5b189278>

### Fit and compile the model with batch_size = 128 and epochs = 10 and execute the model

Try to get training and validation accuracy to be more than 90%

In [0]:
from keras.optimizers import Adam
head_model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['acc'])

In [0]:
history = head_model.fit(x_tr, y_tr,
batch_size=128,
epochs=10,
verbose=1,
validation_data=(x_cv, y_cv))
score = head_model.evaluate(x_cv, y_cv, verbose=0)

Train on 4006 samples, validate on 994 samples
Epoch 1/10
4006/4006 [==============================] - 12s 3ms/step - loss: 4.4928 - acc: 0.0671 - val_loss: 3.9121 - val_acc: 0.1771
Epoch 2/10
4006/4006 [==============================] - 12s 3ms/step - loss: 3.2013 - acc: 0.2713 - val_loss: 2.8040 - val_acc: 0.3089
Epoch 3/10
4006/4006 [==============================] - 12s 3ms/step - loss: 2.2302 - acc: 0.4324 - val_loss: 2.1534 - val_acc: 0.4205
Epoch 4/10
4006/4006 [==============================] - 12s 3ms/step - loss: 1.7481 - acc: 0.5237 - val_loss: 2.0132 - val_acc: 0.4487
Epoch 5/10
4006/4006 [==============================] - 12s 3ms/step - loss: 1.4793 - acc: 0.5946 - val_loss: 1.8268 - val_acc: 0.4889
Epoch 6/10
4006/4006 [==============================] - 12s 3ms/step - loss: 1.2779 - acc: 0.6465 - val_loss: 1.7934 - val_acc: 0.4909
Epoch 7/10
4006/4006 [==============================] - 12s 3ms/step - loss: 1.1510 - acc: 0.6730 - val_loss: 1.7295 - val_acc: 0.5060
Epoch 8/